# Summarizing Steam Reviews

## Pulling in the Data

In [1]:
import pandas as pd

raw_data = pd.read_csv('/kaggle/input/steam-reviews/dataset.csv')

Since I'm just going to be summarizing the review text the extra columns are unnecessary.

In [2]:
raw_data.drop(columns=['review_votes', 'review_score', 'app_id'], inplace=True, errors='ignore')

In [3]:
raw_data.head(5)

,app_name,review_text
0,Counter-Strike,Ruined my life.
1,Counter-Strike,This will be more of a ''my experience with th...
2,Counter-Strike,This game saved my virginity.
3,Counter-Strike,• Do you like original games? • Do you like ga...
4,Counter-Strike,"Easy to learn, hard to master."


In [4]:
raw_data.dropna(axis='index', inplace=True)

In [5]:
reorder_raw_data = raw_data.review_text.str.len().sort_values(ascending=False).index
raw_data = raw_data.reindex(reorder_raw_data)

In [6]:
raw_data.head(5)

,app_name,review_text
1269648,Quake III Arena,Excellent.. (Quake Champions Hype!!!) &nbsp;&...
5071378,Moonbase Alpha,MOONBASE ODDITY REMINDER: EACH LINE IS TOO L...
5837082,Borderlands 2,&lt;10:37:20&gt; Trying to connect to server o...
6035152,Dota 2,"&gt; Good game, well played &gt; Good game, we..."
88380,Terraria,https://www.youtube.com/watch?v=cH6qR2gliug ...


In [7]:
from datasets import Dataset
# Filter only the longest 1,000 reviews to summarize
data_df = Dataset.from_pandas(raw_data[:1000])

In [8]:
import gc
del raw_data
gc.collect()

51

## Model Time

In [9]:
# from transformers import PegasusForConditionalGeneration, PegasusTokenizerFast

# checkpoint = "google/pegasus-xsum"
# model = PegasusForConditionalGeneration.from_pretrained(checkpoint)
# tokenizer = PegasusTokenizerFast.from_pretrained(checkpoint)

In [10]:
# batch = tokenizer(data_df['review_text'], truncation=True, padding='longest', return_tensors="pt")
# translated = model.generate(**batch)
# tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
# data_df['summarized'] = tgt_text

In [11]:
# data_df.head(5)